# Swiss Communes Data Analysis

**Author:** Mame Faty Lo  
**Context:** Applied Data Science: Machine Learning (EPFL Extension School)

---

## Academic Integrity Declaration

Academic Integrity and Learning Statement

By submitting my work, I confirm that:

1. The code, analysis, and documentation in this notebook are my own work and reflect my own understanding.
2. I am prepared to explain all code and analysis included in this submission.

If I used assistance (e.g., AI tools, tutors, or other resources), I have:

- Clearly documented where and how external tools or resources were used in my solution.
- Included a copy of the interaction (e.g., AI conversation or tutoring notes) in an appendix.

I acknowledge that:

- I may be asked to explain any part of my code or analysis during evaluation.
- Misrepresenting assisted work as my own constitutes academic dishonesty and undermines my learning.

## A. Overview of the Dataset

The dataset `p1_communes.csv`, sourced from the Swiss Federal Statistical Office (FSO), is reviewed prior to exploration.

The inspection includes:

- **Structural Validation:** Load the dataset, report the number of rows and columns, and preview the first entries.
- **Schema Profiling:** Examine the index labels, column names, and data types.
- **Completeness & Consistency Audit:** Identify and address missing or inconsistent values.

> Establishing structural clarity and data integrity is a prerequisite for reliable statistical and exploratory analysis.

### Structural Validation (A1–A3)

The dataset is ingested using `pd.read_csv()` to establish the primary DataFrame. The filename is parameterized to maintain modularity and facilitate future updates if the source file changes.

Initial checks using `.head()` and `.shape()` serve as a dimensional check to confirm successful loading, verify expected row and column counts, and ensure proper column alignment before further processing.

In [1]:
import pandas as pd
import numpy as np

# A1. Load dataset
dataset_name = 'p1_communes.csv'
df = pd.read_csv(dataset_name)

# A2. Report the number of rows and columns
print(f'Loaded dataset: {dataset_name} | {df.shape[0]:,} rows x {df.shape[1]} columns')

# A3. Preview the first entries
display(df.head())

Loaded dataset: p1_communes.csv | 2,202 rows x 17 columns


,Canton,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation
0,ZH,Aeugst am Albis,de,1982,250.568900,19.677094,62.764884,17.558022,835,7.91,12.658228,51.139241,30.886076,5.316456,2679300,1235700,673
1,ZH,Affoltern am Albis,de,12229,1154.768650,20.508627,61.329626,18.161747,5348,10.59,30.674264,40.170940,28.205128,0.949668,2676800,1236800,502
2,ZH,Bonstetten,de,5548,746.702557,23.666186,60.310022,16.023792,2325,7.43,15.456989,55.510753,28.629032,0.403226,2677800,1241000,583
3,ZH,Hausen am Albis,de,3701,272.132353,21.804918,60.686301,17.508781,1546,13.60,12.692590,55.906090,28.833456,2.567865,2682900,1233100,653
4,ZH,Hedingen,de,3734,571.822358,21.772898,61.756829,16.470273,1540,6.53,19.817073,46.341463,33.231707,0.609756,2676400,1239000,543


**Initial Observations**

The dataset can be organized into four primary thematic domains:

**1. Administrative and Governance**  
`Canton`, `Commune` establish the administrative hierarchy and provide the basis for regional aggregation, comparison, and grouping analyses.

**2. Sociodemographic**  
`Residents`, `Population density per km²`, age distribution (`0–19`, `20–64`, `65+`), `Language` characterize population size, structure, and demographic composition across communes.

**3. Spatial and Geographic**  
`Surface area (km²)`, `Elevation`, coordinates (`East`, `North`) define the physical attributes of each commune, enabling spatial analysis and geographic contextualization.

**4. Land Use**  
`Settlement`, `Agricultural`, `Wooded`, `Unproductive` quantify land allocation by usage category, supporting proportional and structural land distribution analysis.

### Schema Validation (A4–A5)

The dataset schema is inspected and columns are organized by data type. Columns are then classified to ensure that subsequent preprocessing, aggregation, visualization, and statistical analyses can be applied correctly and efficiently.

In [2]:
# A4. Obtain the index labels and show column names as a list
index_labels = df.index
column_names = df.columns.tolist()
print(f"Index labels: {index_labels}\n\nColumn names: {column_names}\n")

# A5. Examine the data types of each column
display(df.dtypes)

Index labels: RangeIndex(start=0, stop=2202, step=1)

Column names: ['Canton', 'Commune', 'Language', 'Residents', 'Population density per km²', '0-19 years', '20-64 years', '65 years or over', 'Private households', 'Surface area in km²', 'Settlement area', 'Agricultural area', 'Wooded area', 'Unproductive area', 'East coordinate', 'North coordinate', 'Elevation']



Canton                         object
Commune                        object
Language                       object
Residents                       int64
Population density per km²    float64
0-19 years                    float64
20-64 years                   float64
65 years or over              float64
Private households              int64
Surface area in km²           float64
Settlement area               float64
Agricultural area             float64
Wooded area                   float64
Unproductive area             float64
East coordinate                 int64
North coordinate                int64
Elevation                       int64
dtype: object

All columns are stored properly, and data types are consistent with their logical representations (e.g., integers for counts, floats for measurements).

Columns are programmatically grouped into **Quantitative** and **Categorical** types to ensure mathematical transformations only target numerical fields, while grouping operations are applied to categorical identifiers.

In [3]:
# Organize columns by data type for efficient referencing in subsequent tasks
quant_cont = df.select_dtypes(include='float64').columns.tolist()
quant_disc = df.select_dtypes(include='int64').columns.tolist()
cat_nom    = df.select_dtypes(include='object').columns.tolist()

# Create a summary for clear data type mapping
column_summary = pd.DataFrame({
    'Quantitative (Continuous)': pd.Series(quant_cont),
    'Quantitative (Discrete)': pd.Series(quant_disc),
    'Categorical (Nominal)': pd.Series(cat_nom)
}).fillna('')

display(column_summary)

,Quantitative (Continuous),Quantitative (Discrete),Categorical (Nominal)
0,Population density per km²,Residents,Canton
1,0-19 years,Private households,Commune
2,20-64 years,East coordinate,Language
3,65 years or over,North coordinate,
4,Surface area in km²,Elevation,
5,Settlement area,,
6,Agricultural area,,
7,Wooded area,,
8,Unproductive area,,


### Completeness & Consistency Audit (A6–A7)

To ensure all subsequent analyses and aggregations are based on a complete and logically consistent data, the following checks were performed:

1. **Categorical Integrity**  

   - **Duplicates:** Checked for repeated `Commune` entries to eliminate overrepresentation.
   - **Standardisation:** We establish a standardised formatting and case sensitivity for each categorical column.
   
2. **Missing Values**  

Missing entries were identified, quantified, and remediated.

3. **Logical Constraints**

   - **Numerical Validity:** 
     - percentage sums
     - Inspected zero or negative values.  
   - **Relational & Spatial Constraints:**  
     - `Population density per km²` = `Residents`/`Surface area in km²` 
     - `Private households` < `Residents`
     - Coordinates are within official Swiss boundaries  
     - Elevation values are within realistic Swiss ranges

#### 1. Categorical Integrity

##### Duplicates
Each row in the dataset is expected to represent a unique Swiss commune.

In [4]:
# Check for duplicate communes
duplicate_communes = df[df.duplicated(subset='Commune', keep=False)]

# Display count and affected rows
print(f"Number of duplicate commune entries: {len(duplicate_communes)}")
display(duplicate_communes)

Number of duplicate commune entries: 0


,Canton,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation


> No duplicate communes were found, confirming that each row uniquely represents a single commune.

##### Standardisation

All `Commune` strings must be stripped of leading and trailing invisible spaces.

In [5]:
# Ensure Commune column exists and strip leading/trailing spaces
if 'Commune' in df.columns:
    df['Commune'] = df['Commune'].str.strip()

Unique values in `Canton` and `Language` are sorted and counted. They have a fixed set of valid values across all Switzerland, allowing a feasible check on completeness and consistency. 

In [6]:
# Sort Canton and Language alphabetically for easier inspection
unique_cantons = sorted(df['Canton'].dropna().unique())
unique_languages = sorted(df['Language'].dropna().unique())

# Display values and counts
print(f"Unique Cantons ({len(unique_cantons)}): {unique_cantons}\n")
print(f"Unique Languages ({len(unique_languages)}): {unique_languages}\n")

Unique Cantons (26): ['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH']

Unique Languages (4): ['de', 'fr', 'it', 'rm']



> Commune names have no trailing or leading spaces.  
All 26 cantons and 4 languages of Switzerland are represented and consistently formatted.

#### 2. Missing Values (A6–A7)

##### Identification & Ratio
Rows containing missing values are isolated using Boolean indexing: `.isna()` maps missing entries, while `any(axis=1)` identifies any row with at least one data gap. These rows are stored in `missing_rows`.

To assess the impact of missing data, the proportion of missing entries per column is calculated. Applying `.mean()` to the Boolean DataFrame generated by `.isna()` treats `True` as `1` and `False` as `0`, so the mean along each column (`axis=0`) directly yields the fraction of missing values per column.

In [7]:
# A6. Identify rows containing missing values
missing_rows = df[df.isna().any(axis=1)]

# Print the count of rows affected
print(f'Number of rows with missing values: {len(missing_rows)}')

display(missing_rows)

# A7. Evaluate missing data proportions
missing_proportions = df.isna().mean(axis=0)
display(missing_proportions)

Number of rows with missing values: 11


,Canton,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation
155,ZH,Stammheim,NaN,2747,114.649416,21.405169,58.281762,20.313069,1125,23.96,9.056761,55.801336,34.557596,0.584307,2702400,1276500,455
156,ZH,Wädenswil,NaN,24341,682.968575,19.740356,59.944127,20.315517,10371,35.64,19.336706,63.265880,15.064643,2.332771,2693400,1231600,641
157,ZH,Elgg,NaN,4903,201.025010,20.762798,60.473180,18.764022,2121,24.39,9.545268,46.907005,43.138058,0.409668,2707700,1260900,598
158,ZH,Horgen,NaN,22665,735.160558,20.657401,60.480918,18.861681,9685,30.83,18.456049,38.858255,39.506974,3.178722,2687800,1234900,624
440,NaN,Thurnen,NaN,1922,323.025210,20.759625,58.324662,20.915713,814,5.95,13.949580,79.159664,6.050420,0.840336,2605300,1184700,558
704,NaN,Villaz,NaN,2287,148.217758,24.398776,61.346742,14.254482,897,15.44,8.808290,70.336788,20.401554,0.453368,2563200,1174400,727
757,NaN,Prez,NaN,2236,139.401496,25.000000,61.672630,13.327370,839,16.04,6.924517,68.808484,23.268871,0.998129,2567700,1181700,651
1133,GR,Bergün Filisur,NaN,905,4.759651,16.574586,59.668508,23.756906,397,190.14,0.977815,19.824414,27.541794,51.655977,2776700,1166700,2273
1165,GR,Rheinwald,NaN,577,4.217220,16.984402,54.072790,28.942808,266,136.82,1.124334,34.328685,14.448419,50.098562,2744500,1157500,2192
1626,TI,Riviera,NaN,4220,48.780488,20.071090,61.445498,18.483412,1774,86.58,4.008317,7.034770,65.553887,23.403026,2718700,1128900,1458


Canton                        0.001362
Commune                       0.000000
Language                      0.004995
Residents                     0.000000
Population density per km²    0.000000
0-19 years                    0.000000
20-64 years                   0.000000
65 years or over              0.000000
Private households            0.000000
Surface area in km²           0.000000
Settlement area               0.000000
Agricultural area             0.000000
Wooded area                   0.000000
Unproductive area             0.000000
East coordinate               0.000000
North coordinate              0.000000
Elevation                     0.000000
dtype: float64

> **`Canton`** and **`Language`** are the only columns containing very few missing values (approximately 0.14% and 0.50%, respectively). As these are categorical variables that can be easily cross-verified with external sources, removing the affected rows is unnecessary. Imputation preserves information and ensures a more accurate and complete analysis.

##### Imputation via Vectorised Lookup

Missing values in `Canton` and `Language` are resolved using a hand-verified dictionary that maps each commune to its respective values. The `Commune` column identified in A6 serves as a relational lookup key, cross-referenced with external sources such as Google.

In [27]:
# Manual mapping for missing values
lookup = {
    'Stammheim': {'Language': 'de'},
    'Wädenswil': {'Language': 'de'},
    'Elgg':      {'Language': 'de'},
    'Horgen':    {'Language': 'de'},
    'Thurnen':   {'Canton': 'BE', 'Language': 'de'},
    'Villaz':    {'Canton': 'FR', 'Language': 'fr'},
    'Prez':      {'Canton': 'FR', 'Language': 'fr'},
    'Bergün Filisur': {'Language': 'de'},
    'Rheinwald': {'Language': 'de'},
    'Riviera':   {'Language': 'it'},
    'La Grande Béroche': {'Language': 'fr'}
}

# Convert lookup dictionary to a DataFrame for display
lookup_df = pd.DataFrame.from_dict(lookup, orient='index')
display(lookup_df.fillna(''))

,Language,Canton
Stammheim,de,
Wädenswil,de,
Elgg,de,
Horgen,de,
Thurnen,de,BE
Villaz,fr,FR
Prez,fr,FR
Bergün Filisur,de,
Rheinwald,de,
Riviera,it,


The lookup dictionary is iterated over to fill missing values in `Canton` and `Language`. For each commune listed in the dictionary, only the corresponding `NaN` values in the main dataset are updated. Existing non-missing values remain unchanged.

This approach ensures targeted and controlled imputation anchored to the primary key `Commune`, without modifying the DataFrame structure or column order.

After imputation, the dataset is verified to confirm that all missing values in `Canton` and `Language` have been addressed.

**Optional:** To confirm there are no remaining missing values in the entire dataset, `isna()` is chained with a double `sum()` aggregation. The first `sum()` counts nulls per column, and the second aggregates these into a single total for the full dataset.

In [28]:
# Fill missing Canton and Language from dictionary
for commune, values in lookup.items():
    for col, val in values.items():
        df.loc[df['Commune'] == commune, col] = df.loc[df['Commune'] == commune, col].fillna(val)
    
# Verification on affected columns
final_missing = df[['Canton', 'Language']].isna().sum()
if final_missing.sum() == 0:
    print("All missing values in Canton and Language have been filled.")
else:
    print(f"{final_missing.sum()} values still missing.")

# Optional verification on full dataset
total_missing = df.isna().sum().sum()
print(f"Total missing values in dataset: {total_missing}")

All missing values in Canton and Language have been filled.
Total missing values in dataset: 0


#### 3. Logical Constraints

##### Numerical Validity

- **Percentages Sum**

To confirme the internal statistical integrity of each record in percentage, we insvestigate whether

- Age groups (`0-19 years`, `20-64 years`, `65 years or over`) sum to 100% per commune..
- Land use categories (`Agricultural area`, `Wooded area`, `Settlement area`, `Unproductive area`) sum to 100% per commune.  

A float-safe comparison in NumPy (`.isclose`) is used to handle minor numerical precision errors.

In [44]:
import numpy as np

# Sum age groups
age_sum = (
    df['0-19 years'] +
    df['20-64 years'] +
    df['65 years or over']
)

# Sum land use 
land_use = (
    df['Agricultural area'] +
    df['Wooded area'] +
    df['Settlement area'] +
    df['Unproductive area']
)

# Strict 100 check (float-safe)
age_invalid = df[~np.isclose(age_sum, 100)]
land_invalid = df[~np.isclose(land_use_sum, 100)]

# Conditional display
if len(age_invalid) == 0:
    print("All age group sums to 100 for each commune.\n")
else:
    print(f"Age group inconsistencies: {len(age_invalid)}")
    display(age_invalid[['Commune']])

if len(land_invalid) == 0:
    print("All land types sum to 100 for each commune.")
else:
    print(f"Land use inconsistencies: {len(land_invalid)}")
    display(land_invalid[['Commune']])

All age group sums to 100 for each commune.

All land types sum to 100 for each commune.


- **Zeros and Negatives**

All numerical columns are inspected to identify and count zero (`== 0`) and negative values (`< 0`). Only columns with at least one zero or negative value are displayed. If all values are positive and non-zero, a confirmation message is printed. This highlights potential anomalies or data-entry errors.

In [46]:
# Combine continuous and discrete numerical columns
num_cols = quant_cont + quant_disc

# Count zeros and negatives
zero_counts = (df[num_cols] == 0).sum()
negative_counts = (df[num_cols] < 0).sum()

# Display zeros if present
if zero_counts.any():
    print("Number of zero values:")
    print(zero_counts[zero_counts > 0])

# Display negatives if present
if negative_counts.any():
    print("\nNumber of negative values:")
    print(negative_counts[negative_counts > 0])

# Confirmation if all numerical values are positive and non-zero
if not zero_counts.any() and not negative_counts.any():
    print("All numerical values are positive and non-zero.")

Number of zero values:
0-19 years             1
Agricultural area      4
Wooded area            3
Unproductive area    280
dtype: int64


**Demographics**

To systematically validate the one commune with `0-19 years = 0`, the entry is cross-validated against relational benchmarks using the following framework:

*   **Full Demographic Profile:**  
We evaluate the remaining **`20-64`** and **`65 years or over`** age groups to get a sense of the age imbalance.

*   **Correlative Benchmarking:**  
To provide context, we evaluate 
    * **`Residents`**
    * **`Surface Area in km²`**
    * **`Population Density per km²`**
    * **`Elevation`**

We benchmark the commune, **`ZERO Youth`**, against the dataset's absolute extremes:
    * **`MIN Residents`**
    * **`MIN Area`**
    * **`MIN Density`**
    * **`MAX Elevation`**
    
This confirms if it shares the same physical constraints as the most extreme regions in Switzerland.

In [71]:
# Filter communes where `0-19 years = 0` and label them `ZERO Youth`
zero_youth = df[df['0-19 years'] == 0].assign(Benchmark='ZERO Youth')

# Map each benchmark label to the row index of its extreme value in the DataFrame
benchmarks = {
    'MIN Residents': df['Residents'].idxmin(),
    'MIN Area': df['Surface area in km²'].idxmin(),
    'MIN Density': df['Population density per km²'].idxmin(),
    'MAX Elevation': df['Elevation'].idxmax()
}

# Select each extreme row, add its Benchmark label, and merge with Zero-Youth rows
benchmark_rows = [df.loc[[idx]].assign(Benchmark=label) for label, idx in benchmarks.items()]
youth_report = pd.concat([zero_youth] + benchmark_rows, ignore_index=True)

# Select columns to show
cols_to_show = [
    'Benchmark', 'Commune', 'Canton',
    '0-19 years', '20-64 years', '65 years or over',
    'Residents', 'Surface area in km²', 'Population density per km²', 'Elevation'
]

# Display with 1 decimal precision for numerical columns
display(youth_report[cols_to_show].style.format({col: "{:.1f}" for col in num_cols}))

,Benchmark,Commune,Canton,0-19 years,20-64 years,65 years or over,Residents,Surface area in km²,Population density per km²,Elevation
0,ZERO Youth,Corippo,TI,0.0,18.2,81.8,11.0,7.7,1.4,1134.0
1,MIN Residents,Corippo,TI,0.0,18.2,81.8,11.0,7.7,1.4,1134.0
2,MIN Area,Rivaz,VD,19.8,60.1,20.1,358.0,0.3,1154.8,435.0
3,MIN Density,Zwischbergen,VS,5.2,77.9,16.9,77.0,86.0,0.9,2129.0
4,MAX Elevation,Fieschertal,VS,20.1,61.3,18.6,318.0,172.9,1.8,3080.0


**Observations**

* **Dual Record Holder:** **Corippo** holds the national record for both `ZERO Youth` and `MIN Residents`. With only 11 residents, the sample size is too small to guarantee representation in every age bracket.
* **Demographic Profile:** Corippo’s population is overwhelmingly concentrated in the senior demographic, with 81.8% of residents in the `65 years or over` category, the highest proportion in the dataset, indicating a community of retirees.
* **Geographic & Scale Comparison:** While **Zwischbergen (VS)** is more sparsely populated and **Fieschertal (VS)** sits at a much higher elevation (3080 m), both maintain a youth population. Even **Rivaz (VD)**, despite being the smallest commune, sustains a standard youth demographic of ~20%.

> The absence of youth in **Corippo** is a direct consequence of its **critically low population** and **extreme demographic skew**. Otherwise, a youth demographic typically exists regardless of high altitude, low population density, or small surface area.

**Land Use**

Communes reporting 0% for **`Agricultural area`**, **`Wooded area`**, or **`Unproductive area`** are audited using a logic of spatial prioritisation. A null value in any category is not an indicator of missing data, but a result of one land type outcompeting others.

> Zeros in demographic or land-use categories reflect real characteristics of the communes, not missing data. No data rows were removed or modified.

- **Derived Metric Discrepancy Check**

The reported `Population density per km²` must equal the expected density `Residents` ÷ `Surface area in km²` to ensure internal numerical validity. We compute the relative deviation in percent and filters communes where the discrepancy exceeds 2%, highlighting only major inconsistencies. The resulting table is sorted by deviation and numeric values are rounded for clarity.

In [95]:
# Compute expected population density
df['Expected_Density'] = df['Residents'] / df['Surface area in km²']

# Calculate relative deviation in %
df['Density_Deviation'] = (df['Population density per km²'] - df['Expected_Density']).abs() / df['Expected_Density'] * 100

# Filter major discrepancies (>2%) and sort
density_issues = df.loc[df['Density_Deviation'] > 2, 
                             ['Commune', 'Residents', 'Population density per km²', 'Expected_Density', 'Density_Deviation']] \
                        .sort_values('Density_Deviation', ascending=False)

# Display rounded numeric columns
display(density_issues.style.format({col: "{:.2f}" for col in num_cols}))

# Print total count
print(f"Total rows with >2% population density discrepancy: {len(density_issues)}")

,Commune,Residents,Population density per km²,Expected_Density,Density_Deviation
1479,Gottlieben,340.00,1096.77,1062.50,3.23
1599,Ponte Tresa,800.00,1951.22,1904.76,2.44
2039,Grône,2414.00,116.51,118.92,2.03


Total rows with >2% population density discrepancy: 3


> Because the threshold is relative, it accounts for commune size and avoids overstating small rounding differences. The small number of affected communes suggests isolated inconsistencies rather than a systemic data issue. These entries are retained and documented for transparency.

##### Constraints
* **Private households:** Should be less than or equal to `Residents`, reflecting realistic household composition.  
* **Coordinates:** Longitude (`East coordinate`) and latitude (`North coordinate`) must fall within official Swiss boundaries to ensure spatial validity.  
* **Elevation:** Values should be within realistic ranges for Switzerland, confirming plausible topography for each commune.

In [13]:
# Final verification
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Canton                      2199 non-null   object 
 1   Commune                     2202 non-null   object 
 2   Language                    2191 non-null   object 
 3   Residents                   2202 non-null   int64  
 4   Population density per km²  2202 non-null   float64
 5   0-19 years                  2202 non-null   float64
 6   20-64 years                 2202 non-null   float64
 7   65 years or over            2202 non-null   float64
 8   Private households          2202 non-null   int64  
 9   Surface area in km²         2202 non-null   float64
 10  Settlement area             2202 non-null   float64
 11  Agricultural area           2202 non-null   float64
 12  Wooded area                 2202 non-null   float64
 13  Unproductive area           2202 

>The dataset is now clean, consistent, and ready for further analysis.

# B. Exploration: Numerical Summaries, Indexing and Grouping

Following the initial data audit, this section applies targeted aggregation to define the dataset's numerical profiles and regional structures.

**Key Deliverables:**
* **Statistical Benchmarking:** Calculating central tendencies (mean, min, max) for all metrics.
* **Demographic Ranking:** Identifying the most and least populated regional hubs.
* **Cantonal Segmentation:** Evaluating population density and elevation ranges to quantify regional disparities.

## B1. Numerical Summaries

A statistical baseline is established by aggregating the  **mean**, **minimum**, and **maximum** values for all previously identified numerical features in **A5**.

In [31]:
# Concatenate continuous and discrete column lists
num_cols = quant_cont + quant_disc

# Aggregate statistics and transpose for vertical readability
num_summary = df[num_cols].agg(['mean', 'min', 'max']).T

# Display summary rounded to two decimal places
display(num_summary.round(2))

,mean,min,max
Population density per km²,432.45,0.90,12669.05
0-19 years,20.59,0.00,37.78
20-64 years,60.37,18.18,77.92
65 years or over,19.05,6.27,81.82
Surface area in km²,18.16,0.31,438.63
Settlement area,14.86,0.14,97.26
Agricultural area,45.94,0.00,91.51
Wooded area,32.56,0.00,88.24
Unproductive area,6.64,0.00,95.02
Residents,3880.35,11.00,415367.00


## B2. Top 10 most populated communes

We rank communes by the number of residents in descending order to identify the largest populations. We also decided to show their respective cantons for a more rounded analysis.

In [33]:
top_10_most_pop = df.sort_values(by='Residents', ascending=False)[['Commune', 'Canton','Residents']].head(10)
display(top_10_most_pop)

,Commune,Canton,Residents
154,Zürich,ZH,415367
2124,Genève,GE,201818
910,Basel,BS,172258
1749,Lausanne,VD,139111
194,Bern,BE,133883
141,Winterthur,ZH,111851
535,Luzern,LU,81691
1053,St. Gallen,SG,75833
1582,Lugano,TI,63185
207,Biel/Bienne,BE,55159


`df.sort_values(by='Residents', ascending=False)`sorts the entire DataFrame `df` in descending order based on the values in the `'Residents'` column. This ensures the rows with the largest numbers of residents appear first.

After sorting,`[...]` narrows the result to just the data needed for ranking here i.e. `['Commune', 'Residents']`. We also added the `Canton` column for canton-level analysis.

The `.head(10)` method shows the first 10 rows from the sorted DataFrame. Since the DataFrame was sorted in descending order by population, these correspond to the 10 most populated communes.

> **Zurich (ZH)** is the most populated commune in Switzerland, followed by **Geneva (GE)**, **Basel (BS)**, **Lausanne (VD)**, and **Bern (BE)**.

## B3. Top 10 Least Populated Communes
Similarly, we rank communes in ascending order by the number of residents to identify the smallest populations.

In [32]:
top_10_least_pop = df.sort_values(by='Residents')[['Commune', 'Canton', 'Residents']].head(10)
display(top_10_least_pop)

,Commune,Canton,Residents
1546,Corippo,TI,11
875,Kammersrohr,SO,28
2011,Bister,VS,33
366,Schelten,BE,36
370,Rebévelier,BE,41
485,Berken,BE,45
1629,Cerentino,TI,45
1160,Lohn (GR),GR,46
342,Clavaleyres,BE,46
1627,Bosco/Gurin,TI,46


Since ascending order is the default, there is no need to specify `ascending=True`. Because the smallest values appear first, these correspond to the 10 least populated communes.

> **Corippo (TI)** is the least populated commune in Switzerland, followed by **Kammersrohr (SO)**, **Bister (VS)**, **Schelten (BE)**, and **Rebévelier (BE)**.

## B4. Grouping Communes by Canton

We group the dataset by the unique entries in the `Canton` column and create a separate file for each canton.

In [34]:
for canton, communes_in_canton in df.groupby('Canton'):
    # Dro the Canton column
    communes_in_canton = communes_in_canton.drop(['Canton'], axis=1)
    
    # Save to CSV
    communes_in_canton.to_csv(f'{canton}.csv', index=False)
    
    # Print the canton name and number of communes
    print(f"Canton: {canton} | Communes: {len(communes_in_canton)}")
    
    # Display 1st 3 rows for inspection
    display(communes_in_canton.head(3))

Canton: AG | Communes: 210


,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation
1233,Aarau,de,21506,1742.787682,17.343997,64.535478,18.120524,10472,12.34,49.190939,11.407767,32.281553,7.119741,2645800,1249000,377
1234,Biberstein,de,1571,383.170732,17.632081,57.670274,24.697645,689,4.10,13.414634,30.975610,50.975610,4.634146,2648500,1251600,479
1235,Buchs (AG),de,7966,1497.368421,20.913884,63.821240,15.264876,3471,5.32,45.864662,11.466165,42.105263,0.563910,2647600,1248700,383


Canton: AI | Communes: 6


,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation
1045,Appenzell,de,5795,343.305687,18.291631,61.156169,20.552200,2486,16.88,11.552133,65.876777,21.563981,1.007109,2749000,1244100,934
1046,Gonten,de,1462,59.118480,22.777018,55.061560,22.161423,543,24.73,6.058158,59.208401,32.592892,2.140549,2744700,1243700,1018
1047,Rüte,de,3648,89.367957,24.369518,58.333333,17.297149,1386,40.82,3.801815,54.746137,31.444690,10.007358,2750500,1243000,1076


Canton: AR | Communes: 20


...

,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation
1947,Brig-Glis,de,13058,346.733935,18.080870,61.111962,20.807168,5734,37.67,11.677282,12.075372,48.434183,27.813163,2642400,1129600,1416
1948,Eggerberg,de,335,55.371901,13.134328,65.373134,21.492537,150,6.04,5.911330,15.106732,62.561576,16.420361,2634000,1128900,1220
1949,Naters,de,10075,68.434995,18.441687,60.833747,20.724566,4503,147.22,1.825461,17.698154,13.816504,66.659881,2642300,1130700,2428


Canton: ZG | Communes: 11


,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation
657,Baar,de,24468,984.627767,20.181462,63.098741,16.719797,10598,24.85,22.042794,52.402099,24.828421,0.726686,2682600,1227800,521
658,Cham,de,16723,943.203610,21.012976,63.780422,15.206602,6988,17.73,24.831081,60.641892,12.500000,2.027027,2677400,1226100,427
659,Hünenberg,de,8808,478.435633,22.411444,60.104450,17.484105,3477,18.41,14.347826,60.108696,16.630435,8.913043,2674900,1225200,398


Canton: ZH | Communes: 162


,Commune,Language,Residents,Population density per km²,0-19 years,20-64 years,65 years or over,Private households,Surface area in km²,Settlement area,Agricultural area,Wooded area,Unproductive area,East coordinate,North coordinate,Elevation
0,Aeugst am Albis,de,1982,250.568900,19.677094,62.764884,17.558022,835,7.91,12.658228,51.139241,30.886076,5.316456,2679300,1235700,673
1,Affoltern am Albis,de,12229,1154.768650,20.508627,61.329626,18.161747,5348,10.59,30.674264,40.170940,28.205128,0.949668,2676800,1236800,502
2,Bonstetten,de,5548,746.702557,23.666186,60.310022,16.023792,2325,7.43,15.456989,55.510753,28.629032,0.403226,2677800,1241000,583


`df.groupby('Canton')` splits the DataFrame `df` into multiple groups based on the unique values in the `Canton` column. Each group is a subset of the original DataFrame containing all the data for the communes that belong to a single canton.

The `for` loop iterates over these groups and yields two elements at each iteration:  
- `canton`: the value of the `Canton` column for that group (e.g., `GE`).  
- `communes_in_canton`: a DataFrame containing all communes in that canton.

The method `.to_csv()` writes each canton-specific DataFrame to a CSV file.

The filename is created using an f-string, `f'{canton}.csv'`, which dynamically inserts the value of the variable `canton` so that one file is generated per canton (e.g., `GE.csv`). `index=False` ensures that the DataFrame’s index is not included as a separate column in the saved CSV file.

We displayed inside the loop to quickly inspect the first three rows of each canton during iteration.

## B5. Population Density by Canton

We compute the total population and total surface area per canton to calculate population density:

$$
\text{Cantonal population density} = \frac{\text{Total residents}}{\text{Total surface area}}
$$

Cantons are then ranked from the most dense to the least dense.

In [ ]:
canton_density = (
    df.groupby('Canton', as_index=False)
    .agg(
        total_residents=('Residents', 'sum'),
        total_surface=('Surface area in km²', 'sum')
    )
    .assign(pop_density=lambda x: x['total_residents'] / x['total_surface'])
    .sort_values('pop_density', ascending=False)
)

canton_density.round(2)

The DataFrame was grouped by `Canton` (with `as_index=False` to keep the canton names as a column) and the `Residents` and `Surface area in km²` columns were summed using a `.agg()` call. Within .agg(), we renamed the resulting summary columns to `total_residents` and `total_surface`.

A new column, `pop_density`, was then created using `.assign()` with a `lambda` function that divides `total_residents` by `total_surface`. 

The resulting DataFrame was sorted in descending order of `pop_density` to highlight the densest cantons.

To improve readability, the `.round()` method was applied but only during displaying as a way to preserve the high-precision decimals in the `pop_density` variable. 

### Observations

The cantons of **Basel-Stadt** (BS), **Genève** (GE), and **Zürich** (ZH) exhibit the highest population densities in Switzerland, consistent with them containing the most populated communes (B2). Basel-Stadt maintains the highest density primarily due to its exceptionally small land area. While Zürich has the largest population, its significantly larger territory places it behind Basel-Stadt and Geneva in density rankings.

The canton of **Graubünden** (GR) has the lowest population density despite having a larger total population than the densest canton (BS). This is because it occupies the largest geographic area in Switzerland. 

The canton of **Berne** (BE) ranks 19th in density. This cantonal average is misleading, as it aggregates the highly populated communes (B2) with the sparsely ones (B3), hiding significant internal geographic variance.

## B6. Communes with 20–64 Age > 50%
We count communes where more than 50% of the population is aged 20–64 years. We also calculated 

In [ ]:
communes_20_64 = (
    df.groupby('Canton')['20-64 years']
      .agg(
          count_gt_50=lambda x: (x > 50).sum(),
          pct_gt_50=lambda x: (x > 50).mean() * 100
      )
      .sort_values(by='count_gt_50', ascending=False)
      .rename(columns={
          'count_gt_50': 'Communes Count (>50% Aged 20-64)',
          'pct_gt_50': '% of Cantonal Communes'
      })
)

display(communes_20_64)

We first partition the data by `Canton` and isolate the `20-64 years` column.

We called the `.agg()` function to make calculations. Within in it, a `lambda` function was applied to each cantonal group. It evaluates the condition `(x > 50)`, creating a set of boolean values (`True` when the 20-64 age group is more than 50%, `False` otherwise).

3. **Aggregation:**
    * **Counting Communes:** `.sum()` treats `True` as 1 and `False` as 0, adding them up to find the total number of communes that meet the criteria.
    **Proportional Context:** After `.mean()` calculates the average of these 1s and 0s, we multiply by 100 to convert this into a cantonal percentage. This allows for a fairer comparison.
4. **Final Formatting:** The data is sorted by the absolute count using [`.sort_values()`](https://pandas.pydata.org) and then passed through [`.rename()`](https://pandas.pydata.org) to transform the internal logic names into professional titles for the final display.

> Cantons with high administrative counts, such as **Bern** (BE) and **Vaud** (VD), typically show the highest absolute number of communes dominated by the working-age population. However, the percentage column often reveals a different story for smaller or urban cantons like **Basel-Stadt** (BS), where a higher proportion of their total communes may meet this criterion, reflecting [concentrated economic activity](https://www.bfs.admin.ch) and [specific demographic trends](https://www.bfs.admin.ch).


> Cantons with a high total number of communes, such as **Bern** (BE) and **Vaud** (VD), naturally lead in the absolute count of communes where the 20–64 age group exceeds 50%. Conversely, smaller cantons like **Basel-Stadt** (BS) and **Glarus** (GL) show a lower absolute count, reflecting either a different age structure dominating their remaining communes or simply a lower overall number of communes. This distinction between the **absolute count** and the **relative percentage** is crucial for identifying whether a working-age concentration is a localized phenomenon or a canton-wide demographic trend.


## B7. Elevation Range by Canton
We calculate the difference between the maximum and minimum elevation for each canton to identify regions with the largest topographical variation. The top 5 cantons with the largest elevation ranges are reported.

In [ ]:
# Compute elevation range per canton
elevation_range = df.groupby("Canton").apply(
    lambda x: x["Elevation"].max() - x["Elevation"].min()
)

# Sort and take top 5 cantons with largest elevation range
top5_elevation = elevation_range.sort_values(ascending=False).head(5).to_frame(name="Elevation range")

display(top5_elevation)

This code follows the same format as in question B.5 (population density by canton).

First, `df.groupby("Canton")` splits the dataset into groups, one for each canton. 

Then, `.apply(lambda x: x["Elevation"].max() - x["Elevation"].min())` computes the elevation range within each canton by subtracting the minimum elevation from the maximum. 

The `.sort_values(ascending=False)` method orders the cantons from largest to smallest elevation range. 

Using `.head(5)`, we select the top 5 cantons with the most variation in elevation. 

Finally, `.to_frame(name="Elevation range")` converts the resulting Series into a labeled DataFrame for display purposes.

>Cantons like VS (Valais) and TI (Ticino) typically appear at the top because they are **mountainous regions with high elevation differences**. Same for BE (Bern), GR (Graubünden), and UR (Uri).

# C. Exploration: visualizations

Visual exploration helps us understand distributions, patterns, and relationships between variables. It adds depth to our analysis. 

We will use **matplotlib** and **seaborn** to produce informative and clear charts.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Display plots directly inside the notebook
%matplotlib inline

# Remove the grids
sns.set_style("white")

## C1. Horizontal Bar Plot: Top 10 Most Populated Communes

We visualize the top 10 most populated communes from B.2, `top_10_most_pop`, using a horizontal bar chart.

In [ ]:
# Copy top 10 most populated communes
top10 = top_10_most_pop.copy()

# Create a figure
plt.figure(figsize=(8,5))

# Plot horizontal bar chart
sns.barplot(x="Residents", y="Commune", data=top10, palette="viridis")

# Set title and axes labels
plt.title("Top 10 Most Populated Communes in Switzerland")
plt.xlabel("Population")
plt.ylabel("Commune")

# Display plot
plt.show()

`top10 = top_10_most_pop.copy()` creates a copy of the previously computed DataFrame containing the top 10 communes by population. Using `.copy()` ensures that any changes to `top10` do not affect the original DataFrame.


`plt.figure(figsize=(8,5))`creates a new figure object and we control the overall size of the chart. It's only the canvas and the plotting comes next.

`sns.barplot(x="Residents", y="Commune", data=top10)`  takes the DataFrame `top10`and creates a horizontal bar plot where the y-axis shows the commune names and the x-axis shows the population as instructed.  

`palette="viridis"` applies a color gradient to the bars.


`plt.title("...")` adds a title to the chart.

`plt.xlabel("Population")` and `plt.ylabel("Commune")`labels the axes.

`plt.show()` renders the plot in the notebook.

> The chart with the color gradient highlights population concentration in major coummunes across Switzerland. It clearly shows that Zürich significantly dominates all other large urban communes like Geneva, Basel, and Lausanne in terms of population.

## C2. Horizontal Stacked Bar Chart: Division of Land Area of Top 10 Communes

Next, we visualize how land is divided in these top 10 communes among four area types: 
* `Settlement`
* `Agricultural`
* `Wooded`
* `Unproductive`

In [ ]:
# Extract top 10 communes
communes = top_10_most_pop["Commune"].tolist()

# Select the land area columns
top10_area = df.loc[top_10_most_pop.index, ["Settlement area", 
                                            "Agricultural area", 
                                            "Wooded area", 
                                            "Unproductive area"]]

# Assign each land type to a variable for easier calling during stacking
settlement = top10_area["Settlement area"]
agricultural = top10_area["Agricultural area"]
wooded = top10_area["Wooded area"]
unproductive = top10_area["Unproductive area"]

plt.figure(figsize=(10,6))

# Plot stacked horizontal bars
plt.barh(communes, settlement, label="Settlement", edgecolor='none')
plt.barh(communes, agricultural, left=settlement, label="Agricultural", edgecolor='none')
plt.barh(communes, wooded, left=settlement + agricultural, label="Wooded", edgecolor='none')
plt.barh(communes, unproductive, left=settlement + agricultural + wooded, label="Unproductive", edgecolor='none')

plt.xlabel("Area type in %")
plt.ylabel("Commune")
plt.title("10 Most Populated Communes by Type of Area")

# Legend outside the plot
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Fit the plot and legend neatly within the figure boundaries
plt.tight_layout()

plt.show()

We extracted the names of the top 10 most populated communes from the `top_10_most_pop` DataFrame and stored them in a list for the y-axis.

We produced a subset of `df`, `top10_area`, containing the area breakdowns for the top 10 communes. To do so, `df.loc[..., [...]]` selects the four relevant columns corresponding to the four land types for these communes and use the index of `top_10_most_pop` to locate the correct rows.

To create the horizontal stacked bar chart after defining the canvas with `plt.figure()`, we used `plt.barh()` to plot each land type. Stacking is achieved using the `left` parameter, which positions each segment immediately after the previous one. 

Setting `edgecolor='none'` removes the thin white lines that appear between segments by default (see [Matplotlib documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.barh.html?utm_source=chatgpt.com) for `barh()`).

Legend placement was implemented using the `bbox_to_anchor` argument in matplotlib
([Stack Overflow, 2013](https://stackoverflow.com/questions/4700614/how-to-put-the-legend-outside-the-plot)).

> Each horizontal bar represents the total land area of a commune, with the segments showing the composition of land types.


### Analysis of Land Distribution in Top 10 Most Populated Communes

**Highest Settlement Area**
The top 5 most populated communes in Switzerland (Zurich, Geneva, Basel, Lausanne, and Bern) are dominated by settlement area which makes them urbanized.

* Geneva, Basel, and Zurich are **highly urbanized** communes dominated by Settlement area.
* Agricultural, Wooded, and Unproductive land areas are minimal in these specific communes.
* Total land area differences: Given the percentages sum to 100%, some communes like Basel and Geneva appear to have a larger settlement area proportion than Zurich, despite Zurich having a higher population.

even though Luzern has less people than the 6th most populated commune, Winterhur, it has a higher percentage of settlement area

**Semi-Urban Communes: Agriculture and Wooded Balance**
* Luzern, Bern, Lausanne, St. Gallen, and Winterthur retain a noticeable proportion of Agricultural area, indicating coexistence with urban zones.
* Biel/Bienne is unique in this group, as Wooded area is substantially more dominant than agricultural land.
* Lausanne has very little unproductive land.


**Lugano: A Rural-Urban Hybrid**

Lugano is heavily dominated by Wooded area with a substantial Unproductive area. This composition makes sense given its geolocation and natural obstacles (lakes/mountains), leading it to resemble a rural area's land use profile.


**Unproductive Land Overview**

Across all communes, Unproductive land is generally small but visible.
Its presence highlights natural geographic constraints, such as the Alps in Lugano and mountains near Genève.


> **Summary**: Urban areas are largely dominated by Settlement land, while semi-urban communes balance Agricultural and Wooded areas with settlement. Peripheral/rural communes retain more extensive Wooded areas, and Unproductive areas highlight natural geographic constraints. This visualization effectively demonstrates the interplay between population density, urbanization, and underlying land composition.

## C3. Distribution of Young Residents (0-19 yrs)

The histogram below visualizes the distribution of communes based on the proportion of residents aged 0–19.

In [ ]:
plt.figure(figsize=(5, 4))

plt.hist(
    df["0-19 years"],
    bins=[5, 10, 15, 20, 25, 30, 35], # intervals for proportion of young residents
    edgecolor="black" # adds borders to bars for clarity
)

plt.xlabel("Proportion of population aged 0–19 (%)")
plt.ylabel("Number of communes")
plt.title("Distribution of Young Residents (0–19 years) Across Swiss Communes")

plt.show()

Most communes have between 15% and 25% of their population aged 0–19 years. Considering that this age group spans only 20 years (compared to for example 45 years for the 20–64 working-age group), this proportion is consistent with expectations. Very few communes fall below 10% or above 30%, indicating that extreme deviations in the young population are uncommon.

>Overall, this reflects a moderate and reasonable proportion of young residents across Swiss communes.

## C4. Distribution of Young Residents (0–19 years) Across Language Regions

Switzerland has four language regions: 
- german `de`
- french `fr`
- romand `rm`
- italian `it`

To visualize the distribution of the 0–19 age group (**numerical variable**) across those regions (**categorical variable**), seaborn has **three categorical distribution plots** we can choose from: 
- **box**
- **violin**
- **boxen**

Although only one plot type was required, we first display all three side by side to illustrate how different visualization methods reveal complementary aspects of the same distribution.

The centered super title was implemented using the `fig.suptitle` method from the [Matplotlib Figure API](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.suptitle.html).

In [ ]:
# Create a figure with three side-by-side subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

# Use a single color palette for easier comparison of plots
palette = "Set1"

# --- Violin plot (distribution shape + quartiles) ---
sns.violinplot(
    x="Language",
    y="0-19 years",
    data=df,
    palette=palette,
    inner="quartile",
    ax=axes[0]
)

axes[0].set_title("Violin Plot")
axes[0].set_xlabel("Language Region")
axes[0].set_ylabel("Proportion of Population Aged 0–19 (%)")


# --- Box plot (median, IQR, outliers) ---
sns.boxplot(
    x="Language",
    y="0-19 years",
    data=df,
    palette=palette,
    ax=axes[1]
)

axes[1].set_title("Box Plot")
axes[1].set_xlabel("Language Region")
axes[1].set_ylabel("")


# --- Boxen plot (detailed quantiles and tails) ---
sns.boxenplot(
    x="Language",
    y="0-19 years",
    data=df,
    palette=palette,
    ax=axes[2]
)

axes[2].set_title("Boxen Plot")
axes[2].set_xlabel("Language Region")
axes[2].set_ylabel("")


# Shared title for the full figure
fig.suptitle(
    "Distribution of Young Residents (0–19 Years) Across Language Regions",
    fontsize=14,
    y=1.03
)

plt.tight_layout()
plt.show()



> Use a **box plot for clear summary statistics**, a **violin plot to visualize the distribution shape**, and a **boxen plot when a group (e.g., a language region) contains many data points**, so that tail behavior and fine-grained quantiles are meaningful.

We chose the boxen plot with [`stripplot`](https://seaborn.pydata.org/generated/seaborn.stripplot.html#seaborn.stripplot)to see how all the actual observations are distributed within each language region.

This time we used the figure-level interface [`catplot`](https://seaborn.pydata.org/tutorial/categorical.html)

In [ ]:
sns.catplot(
    data=df,
    x="Language",
    y="0-19 years",
    kind="boxen",
    palette="Set1",
    height=6,
    aspect=1.6
)

# Overlay stripplot
sns.stripplot(
    data=df,
    x="Language",
    y="0-19 years",
    color="black",
    size=1.5,       # controls the size of the data points
    alpha=0.9,      # controls their transparency
    jitter=True    # spread them horizontally to avoid overlapping
)

plt.title("Distribution of Young Residents (0–19 Years) Across Language Regions", fontsize=14)
plt.xlabel("Language Region")
plt.ylabel("Proportion of Population Aged 0–19 (%)")

plt.tight_layout()
plt.show()

### Observations

The boxenplot combined with a strip plot effectively visualizes the distribution of young residents (0-19 years) across four language regions (German, French, Romansch, and Italian), revealing data density, clusters, and any extreme observations. Overall, the youth population proportions are mostly between 15% and 25%, but the distributions differ in shape and variability across language regions:  

- **German-speaking (de) communes** have a rather symmetric distribution with slightly shorter whiskers. DE has **both lower and upper outliers**, meaning a few communes have unusually low or high youth proportions. 


- **French-speaking (fr) communes** have the **highest median proportion** and a symmetrical distribution. FR also has **both lower and upper outliers**, mostly below the lower whisker, suggesting a few communes have unusually low youth proportions.  


- **Romansh (rm) communes** show a **left-skewed distribution** with a tall box. This means that although the median proportion is relatively low, many communes have higher proportions of young residents. RM has **no outliers**, so all communes fall within a reasonable range.  

- **Italian (it) communes** are **symmetrical** with a narrower box and **only lower outliers**, indicating that most communes have low to moderate youth proportions, with a few communes slightly below the typical range.  

In summary, the **0–19 population** is generally concentrated around 15–25% across Switzerland. RM communes show **the greatest variability** and a left skew, meaning many have relatively high youth proportions. DE is **the most uniform**, IT shows low to moderate proportions, and FR has **the highest central proportion** of young residents. This analysis highlights not just differences between regions, but the **overall distribution patterns of young populations** across the country.


- The **median proportion of young residents** is similar across all language regions, generally **15–25%**, consistent with expectations given the 20-year span of this group.  
- The **violin plots** reveal that French- and Italian-speaking regions have slightly more variability in the young population, with a few communes above 25% or below 15%.  
- The **boxen plots** highlight extreme values in the tails, but these are rare, confirming that extreme age structures are uncommon.  
- Overall, the 0–19 age distribution is **moderate and relatively balanced across language regions**, with differences primarily due to **natural variation and the shorter span of this age group** compared to 20–64 and 65+.

## C5. Distribution of Age Groups 0-19, 20-64 & 65+ Years Across Language Regions¶

In [ ]:
# Set Seaborn style for nicer plots
sns.set_style("whitegrid")

# Prepare figure and axes: 1 row x 3 columns
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

# Age groups to plot
age_groups = ["0-19 years", "20-64 years", "65 years or over"]

# Loop over axes and age groups
for ax, age in zip(axes, age_groups):
    # Box plot
    sns.boxplot(
        data=df,
        x="Language",
        y=age,
        ax=ax,
        palette="Set2"
    )
    # Strip plot (individual communes as points)
    sns.stripplot(
        data=df,
        x="Language",
        y=age,
        ax=ax,
        color="black",
        alpha=0.3,
        jitter=True
    )
    
    # Titles and labels
    ax.set_title(f"Age group: {age}", fontsize=12)
    ax.set_xlabel("Language region", fontsize=10)
    ax.set_ylim(0, 100)  # Common y-axis across all subplots

# Set y-label only on first subplot
axes[0].set_ylabel("Proportion of population (%)", fontsize=12)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

## C6. Relation Between the Agricultural area of communes and their Settlement area, Wooded area and the Unproductive area.

In [ ]:
# Variables to compare with Agricultural area
y_vars = ["Settlement area", "Wooded area", "Unproductive area"]
x_var = "Agricultural area"

# Create pair plots individually using scatterplots with regression line (optional)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, y_var in zip(axes, y_vars):
    sns.scatterplot(
        data=df,
        x=x_var,
        y=y_var,
        hue="Elevation",      # color code by elevation
        palette="viridis",
        ax=ax
    )
    
    ax.set_xlabel("Agricultural area (%)")
    ax.set_ylabel(f"{y_var} (%)")
    ax.set_title(f"{y_var} vs {x_var}")

# Add legend outside to the right
axes[2].legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Elevation", fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# vertical layout: 1 column (x_vars), 3 rows (y_vars)
g = sns.pairplot(
    data=df,
    x_vars=['Agricultural area'],
    y_vars=['Settlement area', 'Wooded area', 'Unproductive area'],
    hue='Elevation',
    palette='viridis',
    diag_kind=None,  # Fixes the 'missing settlement' rendering bug
    height=4,
    aspect=1.5
)

# Customizing titles/labels to match your task
g.fig.suptitle("Land Use Relations by Elevation", y=1.02)
g.axes[0,0].set_ylabel("Settlement area (%)")
g.axes[1,0].set_ylabel("Wooded area (%)")
g.axes[2,0].set_ylabel("Unproductive area (%)")
g.axes[2,0].set_xlabel("Agricultural area (%)")

plt.show()

In [ ]:
# Set Seaborn style
sns.set_style("whitegrid")

# Create a 1-row, 2-column figure
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Map 1: Color by Elevation ---
sc1 = axes[0].scatter(
    df["East coordinate"],
    df["North coordinate"],
    c=df["Elevation"],
    cmap="terrain",       # green -> brown -> white
    s=40,
    alpha=0.7
)
axes[0].set_title("Swiss Communes by Elevation")
axes[0].set_xlabel("East coordinate")
axes[0].set_ylabel("North coordinate")
cbar1 = plt.colorbar(sc1, ax=axes[0])
cbar1.set_label("Elevation (m)")

# --- Map 2: Color by Language ---
# Fix: use lowercase keys to match df["Language"]
language_palette = {"de": "#1f77b4", "fr": "#ff7f0e", "it": "#2ca02c", "rm": "#d62728"}

axes[1].scatter(
    df["East coordinate"],
    df["North coordinate"],
    c=df["Language"].map(language_palette),
    s=40,
    alpha=0.7
)
axes[1].set_title("Swiss Communes by Language")
axes[1].set_xlabel("East coordinate")
axes[1].set_ylabel("North coordinate")

# Create manual legend for languages
for lang, color in language_palette.items():
    axes[1].scatter([], [], color=color, label=lang, s=40)
axes[1].legend(title="Language", loc="upper right")

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# Set Seaborn style
sns.set_style("whitegrid")

# Create a 1-row, 2-column figure
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Map 1: Color by Elevation ---
sc1 = axes[0].scatter(
    df["East coordinate"],
    df["North coordinate"],
    c=df["Elevation"],
    cmap="terrain",       # green -> brown -> white
    s=40,
    alpha=0.7
)
axes[0].set_title("Swiss Communes by Elevation")
axes[0].set_xlabel("East coordinate")
axes[0].set_ylabel("North coordinate")

# Colorbar as legend for Elevation
cbar1 = plt.colorbar(sc1, ax=axes[0])
cbar1.set_label("Elevation (m)", fontsize=10)
cbar1.ax.tick_params(labelsize=9)

# --- Map 2: Color by Language ---
# Palette keys in lowercase to match df["Language"]
language_palette = {"de": "#1f77b4", "fr": "#ff7f0e", "it": "#2ca02c", "rm": "#d62728"}

axes[1].scatter(
    df["East coordinate"],
    df["North coordinate"],
    c=df["Language"].map(language_palette),
    s=40,
    alpha=0.7
)
axes[1].set_title("Swiss Communes by Language")
axes[1].set_xlabel("East coordinate")
axes[1].set_ylabel("North coordinate")

# Manual legend for languages
for lang, color in language_palette.items():
    axes[1].scatter([], [], color=color, label=lang, s=40)
axes[1].legend(title="Language", loc="upper right")

# Adjust layout
plt.tight_layout()
plt.show()


**References**

McKinney, W. (2022). *Python for data analysis* (3rd ed.). O’Reilly Media.

Pandas Development Team. (2026). *Indexing and selecting data*. https://pandas.pydata.org/docs/user_guide/indexing.html